In [ ]:
import glob
import numpy as np
import os

from argparse import ArgumentParser

from lib import training as tr

import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.optimizers import RMSprop

In [ ]:
def get_option():
    a = ArgumentParser()
    a.add_argument('-k', '--kind', choices = ['mini', 'loto6', 'loto7'], required = True)
    a.add_argument('-e', '--epoch', type = int, required = True)
    a = a.parse_args()
    return a

batch_size = 128

a = get_option()
kind = a.kind
epochs = a.epoch

In [ ]:
def create_model(x_shape):
    model = Sequential()
    model.add(Dense(512, activation='relu', input_shape=(x_shape, )))
    model.add(Dropout(0.2))
    model.add(Dense(512, activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(num_classes, activation='softmax'))
    model.summary()
    model.compile(loss='categorical_crossentropy', optimizer = RMSprop(), metrics = ['accuracy'])
    return model

In [ ]:
num_classes = tr.get_num_classes(kind)
x_train, y_train = tr.prepare_data('csv/'+ kind + '_x.csv', 'csv/' + kind + '_y.csv')
x_train = x_train.astype('float32')
x_train /= num_classes
y_train -= 1
y_train = keras.utils.to_categorical(y_train, num_classes)

In [ ]:
file = 'h5/' + kind + '.h5'
model = None
if os.path.exists(file):
    model = keras.models.load_model(file, compile = True)
else:
    model = create_model(len(x_train[0]))

In [ ]:
history = model.fit(x_train, y_train, batch_size = batch_size, epochs = epochs, verbose = 1)

In [ ]:
model.save(file, include_optimizer = True)